In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:

birdy654_cifake_real_and_ai_generated_synthetic_images_path = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images"
shaileshkvr_resnet_pytorch_default_1_path = "/kaggle/input/forgradcam/pytorch/default/1"

print('Data source import complete.')


Data source import complete.


In [4]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39588 sha256=c9ca037dcee8084f33f97687c037e315eb9404db5dce02776a7fb329dc2e5980
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import numpy as np
import cv2
from tqdm import tqdm
import os

In [7]:
def load_model(model_path, device='cuda'):
    model = resnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model
def generate_gradcam_visualizations(
    model_path,
    dataset_path,
    output_dir,
    num_samples=10,
    device='cuda'
):
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load model
    model = load_model(model_path, device)

    # Define target layer for GradCAM
    target_layers = [model.layer4[-1]]
    # Initialize GradCAM
    cam = GradCAM(
        model=model,
        target_layers=target_layers,
    )

    # Setup data loading
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Load dataset
    dataset = torchvision.datasets.ImageFolder(
        root=dataset_path,
        transform=transform
    )

    # Select random samples
    num_samples = min(num_samples, len(dataset))
    indices = torch.randperm(len(dataset))[:num_samples]

    # Process each image
    for idx in tqdm(indices, desc="Generating Grad-CAM visualizations"):
        # Get image and label
        img_tensor, label = dataset[idx]
        input_tensor = img_tensor.unsqueeze(0)

        # Get original image for overlay
        img_path = dataset.imgs[idx][0]
        rgb_img = cv2.imread(img_path, 1)[:, :, ::-1]
        rgb_img = cv2.resize(rgb_img, (224, 224))
        rgb_img = np.float32(rgb_img) / 255

        # Generate class activation map
        grayscale_cam = cam(input_tensor=input_tensor, targets=None)
        grayscale_cam = grayscale_cam[0, :]

        # Overlay CAM on original image
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

        # Get model prediction
        with torch.no_grad():
            output = model(input_tensor.to(device))
            pred = output.argmax(dim=1).item()

        # Save the visualization
        output_path = os.path.join(
            output_dir,
            f"gradcam_{idx}_pred{pred}_true{label}.jpg"
        )
        cv2.imwrite(output_path, visualization[:, :, ::-1])


In [10]:
generate_gradcam_visualizations(
        model_path='/kaggle/input/forgradcam/pytorch/default/1/fake_detector.pth',
        dataset_path='/kaggle/input/cifake-real-and-ai-generated-synthetic-images',
        output_dir='gradcam_visualizations',
        num_samples=10,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )

/tmp/ipykernel_30/686023814.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
Generating Grad-CAM visualizations: 100%|██████